In [1]:
#importing the necessary files
import json
import jsonlines            #because GPT3 needs files in jsonlines format
import os                   #to get the env variables from the system
import openai 
import pandas as pd
import octopart_check

In [2]:
# Load your API key from an environment variable named as OPENAI_API_KEY
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
#loading the pipe seperated pre-processed test file
with open(r'txt separated\txt separated\training data\Bill of Materials-D-Muster_Pyramid _V1(Standard).txt', 'r') as file:
    training_data = file.read()

In [5]:
#Convert to GPT3 accessible file format : {"text":"...","metadata":"..."}
format_training_data = { "text" : training_data, "metadata" : ""}

In [6]:
#convert the output into jsonl
with jsonlines.open(r'Testing/format_training_data.jsonl', 'w') as writer:
    writer.write(format_training_data)

In [7]:
# create file and uploading
openai.File.create(file=open(r'Testing/format_training_data.jsonl'), purpose='answers')

Upload progress: 100%|███████████████████████████████████████████████████████████| 7.93k/7.93k [00:00<00:00, 14.8kit/s]


<File file id=file-yNPq5TREDpmQ8OzAUX8hHTk9 at 0x1e839a12e28> JSON: {
  "bytes": 7660,
  "created_at": 1618607848,
  "filename": "Testing/format_training_data.jsonl",
  "id": "file-yNPq5TREDpmQ8OzAUX8hHTk9",
  "object": "file",
  "purpose": "answers",
  "status": "uploaded",
  "status_details": null
}

In [10]:
#retrieve information to check if the file was successfully uploaded
openai.File.retrieve('file-yNPq5TREDpmQ8OzAUX8hHTk9')

<File file id=file-yNPq5TREDpmQ8OzAUX8hHTk9 at 0x1e839a5ba08> JSON: {
  "bytes": 7660,
  "created_at": 1618607848,
  "filename": "Testing/format_training_data.jsonl",
  "id": "file-yNPq5TREDpmQ8OzAUX8hHTk9",
  "object": "file",
  "purpose": "answers",
  "status": "processed",
  "status_details": null
}

In [114]:
#Creating parameters for GPT3 to learn our custome DATA
example_text = "|025|2.0|Stk||RS-422/RS-485 Interface IC Half-Duplex RS-485/RS-422-Compatible Transceiver with AutoDirection Control|PCB|MAX13487EESA+|U1, U13|Mouser|Maxim Integrated|https://www.mouser.de/ProductDetail/Maxim-Integrated/MAX13487EESA%2b?qs=sGAEpiMZZMuXae9YOZoWd9EBnNihOkMOLMC5ITnTDKk%3D |045|1.0|Stk||Crystals 32.768kHz 3pF 20ppm -40C +125C|PCB|ABS07W-32.768kHz-K-2-T|Y4|Mouser|ABRACON|https://www.mouser.de/ProductDetail/ABRACON/ABS07W-32768kHz-K-2-T?qs=gt1LBUVyoHkCHJuAeHVlqA%3D%3D |065|8.0|Stk||100uF - 25GV|PCB|865060445005|C11, C15, C16, C25, C27, C36, C38, C72|Würth Electronics||https://www.we-online.de/katalog/de/WCAP-ASLL/?sq=865060445005#865060445005 mV|TAIWAN SEMICONDUCTOR|SK56C||1.0|Farnell|1299292|124156.0|124156.0|| |40.0|Fitted|1.0|U7|FT234XD-R|FT234XD-R - DFN12|DFN-12|FTDI|FT234XD USB to BASIC UART IC, -40 to +85 degC, 12-Pin DFN, Pb-Free, Tape and Reel|FT234XD-R||Mouser|895-FT234XD-R|E00000000144 76.0|IC1|1.0|IFX1117MEV33|Spannungsregler|SOT230P700X170-4N|Infineon Technologies 1.0|U9|DS2764|DS2764BE+025 or DS2764AE+025|Li+ battery monitor I2C bus|Dallas||||75.0|90.0|| 20.0|16.0|STCK|032.062|74HC595 16P-DHVQFN16 HC-MOS SMD|A1-8, A10, A11, A14, A15, A18, A19, A21, A22|Nexperia 74HC595BQ,115 120.0|2.0|STCK|127.001|C 100p 50V NP0 0402 5\%|SMD-KONDENSATOR|C43, C44|32.21.77, Sams CL05C101JB5NNNC"
q_a_examples =[["|025|2.0|Stk||RS-422/RS-485 Interface IC Half-Duplex RS-485/RS-422-Compatible Transceiver with AutoDirection Control|PCB|MAX13487EESA+|U1, U13|Mouser|Maxim Integrated|https://www.mouser.de/ProductDetail/Maxim-Integrated/MAX13487EESA%2b?qs=sGAEpiMZZMuXae9YOZoWd9EBnNihOkMOLMC5ITnTDKk%3D","MAX13487EESA+"],
              ["|045|1.0|Stk||Crystals 32.768kHz 3pF 20ppm -40C +125C|PCB|ABS07W-32.768kHz-K-2-T|Y4|Mouser|ABRACON|https://www.mouser.de/ProductDetail/ABRACON/ABS07W-32768kHz-K-2-T?qs=gt1LBUVyoHkCHJuAeHVlqA%3D%3D","ABS07W-32.768kHz-K-2-T"],
              ["|065|8.0|Stk||100uF - 25GV|PCB|865060445005|C11, C15, C16, C25, C27, C36, C38, C72|Würth Electronics||https://www.we-online.de/katalog/de/WCAP-ASLL/?sq=865060445005#865060445005","865060445005"],
              ["D1|SK56C||SMC|Gleichrichterdiode, Miniatur, Einfach, 60 V, 5 A, DO-214AB, 2, 750 mV|TAIWAN SEMICONDUCTOR|SK56C||1.0|Farnell|1299292|124156.0|124156.0||","SK56C"],
              ["|40.0|Fitted|1.0|U7|FT234XD-R|FT234XD-R - DFN12|DFN-12|FTDI|FT234XD USB to BASIC UART IC, -40 to +85 degC, 12-Pin DFN, Pb-Free, Tape and Reel|FT234XD-R||Mouser|895-FT234XD-R|E00000000144","FT234XD-R"],
              ["76.0|IC1|1.0|IFX1117MEV33|Spannungsregler|SOT230P700X170-4N|Infineon Technologies","IFX1117MEV33"],
              ["1.0|U9|DS2764|DS2764BE+025 or DS2764AE+025|Li+ battery monitor I2C bus|Dallas||||75.0|90.0||","DS2764"],
              ["20.0|16.0|STCK|032.062|74HC595 16P-DHVQFN16 HC-MOS SMD|A1-8, A10, A11, A14, A15, A18, A19, A21, A22|Nexperia 74HC595BQ,115","74HC595BQ,115"],
              ["120.0|2.0|STCK|127.001|C 100p 50V NP0 0402 5%|SMD-KONDENSATOR|C43, C44|32.21.77, Sams CL05C101JB5NNNC","CL05C101JB5NNNC"]] 

In [113]:
#Define the question
Q = "1.0|XT1|14,7456MHz|XJHCCCNANF-14.7456|Quartz SMT; 14,7456MHz|Taitien||20ppm||75.0|90.0||"

In [106]:
#Bring GPT3 into action
openai.Answer.create( 
    model="ada",
    question= Q,
    file="file-yNPq5TREDpmQ8OzAUX8hHTk9", 
    examples_context=example_text,
    examples=q_a_examples,
    max_tokens=15,
    temperature = 0.1,
    stop=["\n", "<|endoftext|>"]
)

<OpenAIObject answer at 0x1e83b5db6a8> JSON: {
  "answers": [
    "XJHCCCNANF-14.7456"
  ],
  "completion": "cmpl-2pS5lM0cOHc8piWcL5too9VRGaBoW",
  "file": "file-yNPq5TREDpmQ8OzAUX8hHTk9",
  "model": "ada:2020-05-03",
  "object": "answer",
  "search_model": "ada:2020-05-03",
  "selected_documents": [
    {
      "document": 23,
      "object": "search_result",
      "score": 66.564,
      "text": "|15.0|Fitted|1.0|LD2|19-337 SMD-RGB-LED|RGB-LED - M1616|M1616|Everlight|0.35mm Height Chip LED with Full Color - SMD|RGB-LED - M1616||Digi-Key|19-337/R6GHBHC-A01/2T-ND|412019337601"
    },
    {
      "document": 12,
      "object": "search_result",
      "score": 68.064,
      "text": "|4.0|Fitted|1.0|C5|Chip-Kerko  22nF/ 6V/ 10% / X5R / 0201|22nF - 0201|0201|MURATA|Capacitor bipolar|22nF - 0402||||21L210411220"
    },
    {
      "document": 4,
      "object": "search_result",
      "score": 68.429,
      "text": "|17.0|Fitted|1.0|LD4|16-216/T3D-AQ1R2TY/3T|LED white - 0402|||LED 0402|LED wh

## Automate the process for every row in the excel sheet provided
Excel file -> pre-process and convert to csv -> take every line in csv as the quesetion -> run gpt N times (N = # of rows in the excel) -> get the answer -> feed into validator function -> output next to the excel as new column 

###### def pre_process_excel()
###### def get_question_from_csv():
###### def run_gpt(question):
###### def validator(ans_from_gpt)
###### def print_to_excel(ans_from_octopart)

In [55]:
#Getting questions from txt file
with open(r'txt separated\txt separated\training data\Bill of Materials-D-Muster_Pyramid _V1(Standard).txt', 'r') as f:
    row = f.readlines()

In [112]:
def get_question_from_csv():
    ques = []
    for i in (row):
        if len(i) > 50 :
            ques.append(i)
    return ques

In [111]:
q = get_question_from_csv()

def run_gpt(q):
    
#     for i in q:
#         response = openai.Answer.create( 
#         model="ada",
#         question= i,
#         file="file-yNPq5TREDpmQ8OzAUX8hHTk9", 
#         examples_context=example_text,
#         examples=q_a_examples,
#         max_tokens=20,
#         stop=["\n", "<|endoftext|>"]
#     )
    for i in q:
        print(i)
        
    ##get the list of answers appended together or indivisualy for every return  
    MPN = response['answers'][0]
    
    return MPN

In [108]:
def validate_MPN(MPN):
    match = demo_match_mpn(client,str(MPN))
    return match

In [116]:
def create_excel_file():
    """can use pandas or xlsxwriter"""
    pass
    return new_excel_file

In [109]:
def print_to_excel(match,new_excel_file):    
    """ get the information from validator and put it in the correct row of newly created excel sheet"""
    
    pass
    print("File is ready to download")